In [1]:
import sys  
sys.path.append("../../")
from LIB.utils import * 
from datetime import timedelta

In [2]:
train=pd.read_csv("../data/train_revised_11.09.2018 (1).zip")
test=pd.read_csv("../data/test_questions (1).csv")

Target=train.groupby(["ride_id"]).seat_number.count().rename("number_of_ticket").reset_index()
train_df=train.drop_duplicates("ride_id").drop(['payment_method', 'payment_receipt', 'seat_number'],axis=1)
train_df=train_df.merge(Target,how="left",on="ride_id")

train_df.drop(["travel_to"],axis=1,inplace=True)
test.drop(["travel_to"],axis=1,inplace=True)

In [3]:
def add_20(x):
    date=x.split("-")
    date[-1]="20"+date[-1]
    return "-".join(date)
train_df["date"]=(train_df["travel_date"].apply(add_20)+" "+train_df["travel_time"]).astype(str)
test["date"]=(test["travel_date"]+" "+test["travel_time"]).astype(str)

def  time_features(x): 
    x["date"]=pd.to_datetime(x["date"],format='%d-%m-%Y %H:%M')
    x["dayofweek"]=x["date"].dt.dayofweek
    x["dayofyear"]=x["date"].dt.dayofyear
    x["dayofmonth"]=x["date"].dt.day
    x["year_woy"]=x["date"].dt.year.astype(str)+x["date"].dt.weekofyear.astype(str)
    x["hour"]=x["date"].dt.hour
    x["minute"]=x["date"].dt.minute
    x["is_weekend"]=x["dayofweek"].apply( lambda x : 1 if x  in [5,6] else 0 )
    x["year"]=x["date"].dt.year
    x["quarter"]=x["date"].dt.quarter
    x["month"]=x["date"].dt.month
    

    return x 
train_df=time_features(train_df)


def  time_features(x): 
    x["date"]=pd.to_datetime(x["date"],format='%Y-%m-%d %H:%M')
    x["dayofweek"]=x["date"].dt.dayofweek
    x["hour"]=x["date"].dt.hour
    x["minute"]=x["date"].dt.minute
    x["dayofyear"]=x["date"].dt.dayofyear
    x["dayofmonth"]=x["date"].dt.day
    x["is_weekend"]=x["dayofweek"].apply( lambda x : 1 if x  in [5,6] else 0 )
    x["year_woy"]=x["date"].dt.year.astype(str)+x["date"].dt.weekofyear.astype(str)
    x["year"]=x["date"].dt.year
    x["quarter"]=x["date"].dt.quarter
    x["month"]=x["date"].dt.month
    return x 
test=time_features(test)

In [4]:
data_characterization(train_df)

shape of data : (6249, 18)


,Columns name,Type,Count unique values,Count Nan values,Ratio Nan values,Unique values,Max,Min,Mean
0,ride_id,int64,6249,0,0.0000,"[1442, 5437, 5710, 5777, 5778, 5781, 5784, 578...",20117,1442,9963.6446
1,travel_date,object,149,0,0.0000,"[17-10-17, 19-11-17, 26-11-17, 27-11-17, 20-04...",0,0,0.0000
2,travel_time,object,78,0,0.0000,"[7:15, 7:12, 7:05, 7:10, 7:09, 5:10, 9:50, 7:0...",0,0,0.0000
3,travel_from,object,17,0,0.0000,"[Migori, Keroka, Homa Bay, Kisii, Keumbu, Rong...",0,0,0.0000
4,car_type,object,2,0,0.0000,"[Bus, shuttle]",0,0,0.0000
5,max_capacity,int64,2,0,0.0000,"[49, 11]",49,11,30.3922
6,number_of_ticket,int64,50,0,0.0000,"[1, 5, 31, 26, 2, 3, 11, 4, 9, 6, 8, 10, 24, 7...",50,1,8.2645
7,date,datetime64[ns],4579,0,0.0000,"[2017-10-17T07:15:00.000000000, 2017-11-19T07:...",0,0,0.0000
8,dayofweek,int64,7,0,0.0000,"[1, 6, 0, 4, 3, 2, 5]",6,0,2.9714
9,dayofyear,int64,149,0,0.0000,"[290, 323, 330, 331, 110, 109, 108, 107, 106, ...",365,1,133.2759


In [5]:
data_characterization(test)

shape of data : (1111, 17)


,Columns name,Type,Count unique values,Count Nan values,Ratio Nan values,Unique values,Max,Min,Mean
0,ride_id,int64,1111,0,0.0000,"[247, 256, 275, 285, 286, 287, 288, 292, 298, ...",15344,247,9166.7075
1,travel_date,object,18,0,0.0000,"[2018-05-07, 2018-05-06, 2018-05-04, 2018-05-0...",0,0,0.0000
2,travel_time,object,79,0,0.0000,"[07:06, 11:08, 05:00, 09:10, 09:20, 07:02, 09:...",0,0,0.0000
3,travel_from,object,15,0,0.0000,"[Kisii, Rongo, Sirare, Homa Bay, Rodi, Awendo,...",0,0,0.0000
4,car_type,object,2,0,0.0000,"[Bus, shuttle]",0,0,0.0000
5,max_capacity,int64,2,0,0.0000,"[49, 11]",49,11,31.4878
6,date,datetime64[ns],744,0,0.0000,"[2018-05-07T07:06:00.000000000, 2018-05-06T11:...",0,0,0.0000
7,dayofweek,int64,7,0,0.0000,"[0, 6, 4, 3, 2, 1, 5]",6,0,2.7372
8,hour,int64,9,0,0.0000,"[7, 11, 5, 9, 19, 8, 6, 10, 23]",23,5,9.2520
9,minute,int64,29,0,0.0000,"[6, 8, 0, 10, 20, 2, 30, 3, 40, 50, 1, 15, 7, ...",59,0,19.1692


In [6]:
test.to_csv("./final_data/proceed_test.csv",index=False)
train_df.to_csv("./final_data/proceed_train.csv",index=False)